In [1]:
# UNIVERSIDADE ESTADUAL DE CAMPINAS
# FT - UNICAMP / CURSO SISTEMAS DE INFORMAÇÃO
#TCC - CORRETOR ORTOGRÁFICO PARA ANALISE DE TIPOS DE ERROS E SUA RELAÇÃO COM O ANALFABETISMO
# AUTOR - MATHEUS EDUARDO DA SILVA 
# ORIENTADORA -ANA ESTELA ANTUNES DA SILVA

In [2]:
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup 
import nltk
#nltk.download('punkt')
import os
import pandas as pd 
from nltk.test.portuguese_en_fixt import setup_module
import nltk.corpus
import re
from collections import Counter

#lendo o conjunto de corpus C2, e salvando seu conteudo em all_texts.txt:
parser = ET.XMLParser(encoding="UTF-8")
file = open("all_texts.txt", "w", encoding="UTF-8")

directory = 'corpus02/ResDialCorpus/C2/'
for filename in os.listdir(directory):
    if filename.endswith('.xml'):
        f = os.path.join(directory, filename)
        tree = ET.parse(f)
        root = tree.getroot()
        text = root.findtext('text')
        file.write(text)

file.close()

In [3]:
#split_words(): recebe uma lista de strings, e cria um novo vetor que contem somente palavras válidas(sem caracteres nao alfanumericos) 
def split_words(tokens_list):
    list_words = []
    for token in tokens_list:
        if token.isalpha():
            list_words.append(token)
    return list_words

#registrando palavras dos resumos e historias
with open('all_texts.txt', encoding='UTF-8') as f:
    contents = f.read()

#registrando palavaras do dicionario em ptbr
with open('dicionario_USP_ptBr.txt', encoding='UTF-8') as f:
    dict_content = f.read()

#tokenizando e criando a lista de palavras dos 3 grupos de palavras(vocabulario, dicionário e resumos)
contents = nltk.tokenize.word_tokenize(contents)
dict_content = nltk.tokenize.word_tokenize(dict_content)
folha_sp = nltk.corpus.mac_morpho.words()

list_words = split_words(contents)
list_words_dict = split_words(dict_content)
list_words_voc_common_words_folhasp = split_words(folha_sp)
print(f"Número de palavras(resumos) é {len(list_words)}")
print(f"Número de palavras(dicionario PTBR) é {len(list_words_dict)}")
print(f"Número de palavras(vocabulario Folha SP) é {len(list_words_voc_common_words_folhasp)}")

Número de palavras(resumos) é 60678
Número de palavras(dicionario PTBR) é 261798
Número de palavras(vocabulario Folha SP) é 1012661


In [4]:
#normalizando as palavras(tornando todas em minúsculas):
def normalization_words(list_words):
    normalized_list = []
    for word in list_words:
        normalized_list.append(word.lower())
    return normalized_list

normalized_list = normalization_words(list_words)
normalized_list_dict = normalization_words(list_words_dict)
normalized_list_voc_folhaSP = normalization_words(list_words_voc_common_words_folhasp)

In [5]:
#valor de frequencia para todas palavras do vocabulário
frequency = nltk.FreqDist(normalized_list_voc_folhaSP)

#numero de palavras no vocabulario(total)
words_total = len(normalized_list_voc_folhaSP)

In [6]:
#Corretor Ortográfico:

#known_words(): retorna a lista de palavras e sua classificação de erro caso existam no dicionário.
def known_words(words): 
    lista = set(w for w in words if w[0] in normalized_list_dict)
    return lista

#candidates_words(): geração de palavras candidatas a possibilidades de correção.
def candidates_words(word):
    candidates = known_words([word]) | known_words(edits_distance_1(word)) | set([word])
    return candidates

#probability():Probabilidade da palavra ser a correte dentre as possíveis.
def probability(word): 
    if isinstance(word, tuple) == True:
        return (frequency[word[0]] /words_total)
    else:
        return (frequency[word]/words_total)

#correction(): função base do corretor, recebe uma palavra e retorna sua maior probabilidade de correção da palavra.
def correction(word):
    result = max(candidates_words(word), key=probability) #, default=word)
    if isinstance(result, tuple) != True:
        return (word, "")
    else:
        return (result)



#Possiblidades de correção de palavras:
    #Grupos de erros(classificação):
    #1- Emprego das consoantes e dos dígrafos + Emprego das formas que representam o som nasal
    #2- Emprego de vogais
    #3- Acréscimo e omissão de letras
    #4- Inversão de letras
    #5- Letras com formato semelhante
    #6- Erros decorrentes de escritas particulares + Segmentação indevida das palavras
    #7- Uso de acentuação

#edit_word_insert(): edição de palavras através de adição.
def edit_word_insert(word_sliced, letters):
    error_group = 3
    error_group_content = []
    new_possible_words = []
    for E, D in word_sliced:
        for letter in letters:
            new_possible_words.append(E + letter + D)
            error_group_content.append(error_group)
    words = list(zip(new_possible_words, error_group_content)) 
    return words

#edit_word_delete(): edição de palavras através de remoção.
def edit_word_delete(word_sliced, letters):
    error_group = 3
    error_group_content = []
    new_possible_words = []
    for E, D in word_sliced:
        if len(D) > 0:
            new_possible_words.append(E + D[1:])
            error_group_content.append(error_group)
    words = list(zip(new_possible_words, error_group_content))
    return words

#edit_word_transpose(): edição de palavras através de transposição.
def edit_word_transpose(word_sliced, letters):
    error_group = 4
    error_group_content = []
    new_possible_words = []
    for E, D in word_sliced:
        if len(D) > 1:
            new_possible_words.append(E + D[1] + D[0] + D[2:])
            error_group_content.append(error_group)
    words = list(zip(new_possible_words, error_group_content))
    return words

#edit_word_replace(): edição de palavras através de substituição.
def edit_word_replace(word_sliced, letters):
    error_group = 0 

    error_group_content = []
    new_possible_words = []
    vogals = ['a', 'e', 'i', 'o', 'u']
    digrafos = ['ch', 'lh', 'nh', 'rr', 'ss', 'sc', 'sç', 'xc', 'xs', 'am', 'an', 'em', 'en', 'im', 'in', 'om', 'on', 'um', 'un', 'gu', 'qu'] #gu e qu somente nos casos de fonema único(sem som "/u/")

    letters_equals_p = ['p', 'f', 'q'] 
    letters_equals_m = ['m', 'n']

    accents = ['á', 'â', 'à', 'ã', 'é', 'ê', 'è', 'ẽ', 'í', 'î', 'ì', 'ĩ', 'ó', 'ô', 'õ', 'ò', 'ú', 'û', 'ù', 'ũ', 'ç']
    
    for E, D in word_sliced:
        if len(D) > 0:
            for letter in letters:
                removed_letter = D[:1]
                new_possible_words.append(E + letter + D[1:]) #letra D[0] é substituida
                if len(E) > 1 and len(D) > 1:
                    possible_digraphs = [(E[-1] + letter), (letter + D[1])]
                    next_letter_digraph = D[1:2]
                elif len(D) > 1 and len(E) <= 1:
                    possible_digraphs = [(E + letter), (letter + D[1])]
                    next_letter_digraph = D[1:2]
                elif len(E) > 1 and len(D) == 1:
                    possible_digraphs = [(E[-1] + letter), (letter)]
                    next_letter_digraph = ""
                else:
                    possible_digraphs = [(E + letter), (letter)]
                    next_letter_digraph = ""

                if ((possible_digraphs[0] in digrafos) or (possible_digraphs[1] in digrafos)) and (next_letter_digraph not in vogals and next_letter_digraph != 'h')  and (letter != removed_letter): #se soma letra + letra anterior ou proxima resulta em digrafo
                    error_group = 1 #digrafos, consoantes ou sons nasais
                elif (((removed_letter in letters_equals_p) and (letter in letters_equals_p)) or ((removed_letter in letters_equals_m) and (letter in letters_equals_m))) and (letter != removed_letter): #erros de formato semelhante
                    error_group = 5
                elif ((((removed_letter not in vogals) and (removed_letter not in accents)) and ((letter not in vogals) and (letter not in accents)) and (removed_letter not in accents))) and (letter != removed_letter): #consoantes incorretas)
                    error_group = 1 #digrafos, consoantes ou sons nasais
                elif ((removed_letter in vogals) and (letter in vogals)) and (letter != removed_letter): #vogais incorretas
                    error_group = 2 #uso de vogais
                elif (((removed_letter in vogals) and (letter in accents)) or (removed_letter == 'c' and letter == 'ç')): #erros de acentuacao
                    error_group = 7 #erro de acentuação
                else:
                    error_group = 6 #erros de escritas particulares
                error_group_content.append(error_group)
    words = list(zip(new_possible_words, error_group_content))
    return words 

#edits_distance_1(): execução das correções na distância de edição 1 para as palavras no corretor.
def edits_distance_1(word):
    word_sliced = []
    letters = 'abcdefghijklmnopqrstuvwxyzáâàãéêèẽíîìĩóôõòúûùũç'
    for i in range(len(word)+1):
        word_sliced.append((word[:i], word[i:]))

    deletes    = edit_word_delete(word_sliced, letters)
    #print(deletes)
    inserts    = edit_word_insert(word_sliced, letters)
    #print(inserts)
    transposes = edit_word_transpose(word_sliced, letters)
    #print(transposes)
    replaces   = edit_word_replace(word_sliced, letters)
    #print(replaces)

    return set(deletes + transposes + replaces + inserts)


In [7]:
#exemplo de uso do corretor ortográfico: recebe uma palavra incorreta, retorna sua correção e a classificação do tipo de erro ocorrido.
correction("somho")

('sonho', 5)

In [8]:
#create_test_dataset(): criador de dados de teste.
def create_test_dataset(file):
    list_words_test = []
    correct = ''
    wrong = ''
    error_group = 0
    f = open(file, "r", encoding='UTF-8')
    for line in f:
        correct, wrong, error_group = line.split(" ", 3)
        list_words_test.append((correct, wrong, error_group))
    f.close()
    return list_words_test

In [9]:
#adjust_format(): ajustar formato das saidas do corretor ortografico.
def adjust_format(word):
    if isinstance(word, tuple) != True:
        return (word, "" )
    else:
        return word

#evaluate_corrections(): avaliador do corretor ortográfico.
    #Grupo WR - correção incorreta e classificação correta
    #Grupo RR - correção correta e classificação correta
    #Grupo WW - correção incorreta e classificação incorreta
    #Grupo RW - correção correta e classificação incorreta
def evalutate_corrections(tests, vocabulary):
    number_words = len(tests)

    right_corrections_right_group = 0
    right_corrections_wrong_group = 0
    wrong_corrections_right_group = 0
    wrong_corrections_wrong_group = 0

    list_RR = []
    list_RW = []
    list_WR = []
    list_WW = []
    list_unknown = []

    unknown = 0
    for correct, wrong, error_group in tests:
        error_group = error_group[0]
        corrected_word = correction(wrong)
        corrected_word = adjust_format(corrected_word)
        result = [corrected_word[0], corrected_word[1], correct, error_group]
        if (correct not in vocabulary):
            list_unknown.append(result)
            unknown += 1
        elif (str(corrected_word[0]) == correct) and (str(corrected_word[1]) == str(error_group)):
            list_RR.append(result)
            right_corrections_right_group += 1 
        elif (str(corrected_word[0]) == correct) and (str(corrected_word[1]) != str(error_group)):
            list_RW.append(result)
            right_corrections_wrong_group += 1 
        elif (str(corrected_word[0]) != correct) and (str(corrected_word[1]) == str(error_group)):
            list_WR.append(result)
            wrong_corrections_right_group += 1
        else:
            list_WW.append(result)
            wrong_corrections_wrong_group += 1

    accuracy_percentage = round(right_corrections_right_group*100/number_words, 2)
    accuracy_percentage_error_group = round((right_corrections_right_group + wrong_corrections_right_group)*100/number_words, 2)
    unknown_percentage = round(unknown*100/number_words, 2)
    print('Grupo WR : ' , list_WR)
    print('Grupo RW : ' , list_RW)
    print('Grupo WW : ', list_WW)
    print('\n')
    print(f"{accuracy_percentage}% de {number_words} palavras, desconhecida é {unknown_percentage}%,  acurácia de erros de grupo {accuracy_percentage_error_group}%")
    print("Palavras Desconhecidas: " + str(len(list_unknown)))
    print("Grupo palavras WW: " + str(len(list_WW)))
    print("Grupo palavras WR: " + str(len(list_WR)))
    print("Grupo palavras RW: " + str(len(list_RW)))
    print("Grupo palavras RR: " + str(len(list_RR)))

    return list_RR, list_RW, list_WR, list_WW, list_unknown

In [10]:
#criando o corpora para teste do corretor:
words_testing_list = create_test_dataset('words_testing.txt')

#executando o avaliador do corretor ortográfico:
list_RR, list_RW, list_WR, list_WW, list_unknown = evalutate_corrections(words_testing_list, normalized_list_voc_folhaSP)

In [ ]:
#criação de dataframes para análises:
columns = ['CorrecaoExecutada', 'ClassificacaoCorrecao', 'CorrecaoDesejada', 'ClassificacaoDesejada']
df_graphics_WR = pd.DataFrame(list_WR, columns=columns)
df_graphics_RW = pd.DataFrame(list_RW, columns=columns)
df_graphics_WW = pd.DataFrame(list_WW, columns=columns)

df_graphics_WR['ClassificacaoCorrecao'].loc[df_graphics_WR['ClassificacaoCorrecao']==""] = 0
df_graphics_RW['ClassificacaoCorrecao'].loc[df_graphics_RW['ClassificacaoCorrecao']==""] = 0
df_graphics_WW['ClassificacaoCorrecao'].loc[df_graphics_WW['ClassificacaoCorrecao']==""] = 0

df_graphics_WW['Grupo(Correcao-Classificacao)'] = 'Errada-Errado'
df_graphics_WR['Grupo(Correcao-Classificacao)'] = 'Errada-Correto'
df_graphics_RW['Grupo(Correcao-Classificacao)'] = 'Correta-Errado'

df_errors = pd.concat([df_graphics_WR, df_graphics_RW, df_graphics_WW])
df_errors

C:\Users\mathe\AppData\Local\Temp\ipykernel_14572\1621461281.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_graphics_WR['ClassificacaoCorrecao'].loc[df_graphics_WR['ClassificacaoCorrecao']==""] = 0
C:\Users\mathe\AppData\Local\Temp\ipykernel_14572\1621461281.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_graphics_RW['ClassificacaoCorrecao'].loc[df_graphics_RW['ClassificacaoCorrecao']==""] = 0


,CorrecaoExecutada,ClassificacaoCorrecao,CorrecaoDesejada,ClassificacaoDesejada,GrupoTipoErro
0,este,1,esse,1,Errada-Correto
1,ele,2,ela,2,Errada-Correto
2,dos,6,das,6,Errada-Correto
3,ele,2,ela,2,Errada-Correto
4,ter,3,terra,3,Errada-Correto
0,universo,1,universo,2,Correta-Errado
1,sonho,5,sonho,1,Correta-Errado
0,em,3,ele,1,Errada-Errado
1,ptyhon,0,python,4,Errada-Errado
2,de,1,ter,3,Errada-Errado
